The dataset we'll be working with is [Bike Share ridership](https://open.toronto.ca/dataset/bike-share-toronto-ridership-data/) data from the City of Toronto Open Data portal.

We can download it and save it in a folder as follows:

In [1]:
import urllib.request

year = 2022
url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/7e876c24-177c-4605-9cef-e50dd74c617f/resource/db10a7b1-2702-481c-b7f0-0c67070104bb/download/bikeshare-ridership-" + str(year) + ".zip"
folder = "data"
urllib.request.urlretrieve(url, folder + "/bike-share-ridership-" + str(year) + ".zip")

FileNotFoundError: [Errno 2] No such file or directory: 'data/bike-share-ridership-2022.zip'

The zip folder has `.csv` data for each month in the selected year. 

Since our data are zipped, we can either unzip the folder manually and run `df = pd.read_csv(path_to_csv_file)`.

Or we can load using the `zipfile` library. I'm feeding in variables for year and month that can easily allow for switching these out or looping over multiple in the future.

In [ ]:
import zipfile

month = '06'

with zipfile.ZipFile("data/bike-share-ridership-" + str(year) + ".zip") as myzip:
    with myzip.open("bikeshare-ridership-" + str(year) + "/Bike share ridership " + str(year) + "-" + month + ".csv") as myfile:
        df = pd.read_csv(myfile)